This is the solution to Fervo Energy Interview video timeseries problem

In [ ]:
#!sudo apt -q install tesseract-ocr
!pip -q install pytesseract

Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 2s (1,950 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Di

In [ ]:
lambda_name        = "VideoProcessing"             # Name of the Lambda function to invoke
results_bucket     = "video-results-obinnak"                    # Bucket to use
results_folder    = "cpu_df"  # Subfolder to place the calculation results
concurrent_lambdas = 100

source_bucket   = 'obinnak-public'    # S3 bucket name with input data
source_folder   = 'data'         # Folder path, sometimes referred to as the prefix
source_filename = 'Test_Video.mp4' # Filename


In [ ]:
# Get S3 object
data_obj = s3.Object(source_bucket, f"{source_folder}/{source_filename}")

# Define some needed variables based off the above parameters
start_time = time.time()

results_file_list = [] # Lets keep track of the output file names, so we can grab them later

lambda_client = boto3.client('lambda')

# Build the message for the Lambda to find the video file
payload = {
    "bucket_in"          : source_bucket,
    "folder_in"          : source_folder,
    "filename_in"        : source_filename,
    "bucket_out"         : results_bucket,
    "folder_out"         : results_folder,
    "is_gpu"         : is_gpu
    }

# Invoke the Lambda SegyBatchProcessMeanAmp
response = lambda_client.invoke(FunctionName=lambda_name,
                                InvocationType='Event',
                                Payload=json.dumps(payload))

results_file_list.append(f"{results_folder}/{source_filename}.pkl")

print("Done!  Elapse time to gather send to and processing on Lambda: {:0.2f} seconds.  Now waiting for 80 seconds.".format(time.time() - start_time))

time.sleep(80)      # Waiting before carrying on next steps, to allow time for Lambda to finish.

In [ ]:
s3_client = boto3.client('s3')
# Get file from S3, convert from Pickle format
object = s3_client.get_object(Bucket=results_bucket, Key=results_file_list[x])

serializedObject = object['Body'].read()
results_bundle_temp = pickle.loads(serializedObject)